In [1]:
import torch.onnx

input_size = (1, 3, 128, 128)
save_path = "../../blazeface/weights/20230823/Blaze_Final.onnx"

#Function to Convert to ONNX
def Convert_ONNX(model):

    # set the model to inference mode 
    model.eval() 

    # Let's create a dummy input tensor  
    dummy_input = torch.randn(input_size, requires_grad=True)  

    # Export the model   
    torch.onnx.export(model,         # model being run 
                    dummy_input,       # model input (or a tuple for multiple inputs) 
                    save_path,       # where to save the model  
                    export_params=True,  # store the trained parameter weights inside the model file 
                    opset_version=12,    # the ONNX version to export the model to 
                    do_constant_folding=True,  # whether to execute constant folding for optimization 
                    input_names = ['Input'],   # the model's input names 
                    output_names = ['Output'], # the model's output names 
                    #  dynamic_axes={'modelInput' : {0 : 'batch_size'},    # variable length axes 
                    #                         'modelOutput' : {0 : 'batch_size'}}
    ) 
    print(" ") 
    print('Model has been converted to ONNX')

In [2]:
from models.net_blaze import Blaze
cfg_blaze = {
    'name': 'Blaze',
    # origin anchor
    'min_sizes': [[16, 24], [32, 48, 64, 80, 96, 128]],
    # kmeans and evolving for 640x640
    #'min_sizes': [[8, 11], [14, 19, 26, 38, 64, 149]], 
    'steps': [8, 16],
    'variance': [0.1, 0.2],
    'clip': False,
    'loc_weight': 1,
    'cls_weight': 6,
    'landm_weight': 0.1, 
    'gpu_train': True,
    'batch_size': 256,
    'ngpu': 1,
    'epoch': 1,
    'decay1': 130,
    'decay2': 160,
    'decay3': 175,
    'decay4': 185,
    #'image_size': 320,
    'image_size': 128,
    'num_classes':2
}

model = Blaze(cfg=cfg_blaze)
load_path = "../../blazeface/weights/20230823/Blaze_Final.pth"
state_dict = torch.load(load_path)
# create new OrderedDict that does not contain `module.`
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    head = k[:7]
    if head == 'module.':
        name = k[7:] # remove `module.`
    else:
        name = k
    new_state_dict[name] = v
model.load_state_dict(new_state_dict)

# Conversion to ONNX 
Convert_ONNX(model)

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

 
Model has been converted to ONNX
